#Enterotype identification and generating Figure 3

In [ ]:
#Import packages
library(phyloseq)
library(vegan)
library(ggplot2)
library(OneR)
library(plyr)
library(dplyr)
library(microbiome)
library(DirichletMultinomial)
library(reshape2)
library(magrittr)

In [ ]:
# Load genus-level rarefied Arivale data
df<-readRDS('rarefied_genus_16S.rds')
#check the phyloseq object
df

In [ ]:
# extract the genus count matrix and convert it into samples x taxa format
dat <- abundances(df)
count <- as.matrix(t(dat))
head(count)
#check dimensions
dim(count)

In [ ]:
#apply dirichlet multinomial modeling with 1-8 clusters
fit <- lapply(1:8, dmn, count=count, verbose=TRUE,seed=1234)

In [ ]:
#plot BIC fit
bic  <- sapply(fit, BIC)
plot(bic, type="b", xlab="Number of Dirichlet Components", ylab="Model Fit")

In [ ]:
lplc <- sapply(fit, laplace) # Laplace

In [ ]:
plot(lplc, type="b", xlab="Number of Dirichlet Components", ylab="Model Fit")

In [ ]:
#choose best number of clusters
best <- fit[[which.min(unlist(bic))]]
best
#assign samples to each enterotype
ass <- apply(mixture(best), 1, which.max)
sample_data(df)$entero<-as.factor(ass)
#save phyloseq object with enterotype classification
saveRDS(df,'entero_df.rds')

In [ ]:
#read phyloseq object with enterotype assignment
df<-readRDS('entero_df.rds')
sample_data(df)<-sample_data(df)[order(sample_data(df)$public_client_id),] 

In [ ]:
#import metadata
metab<-read.csv('statins_df.csv')
#correct id (the zero for some reason gets removed in R)
metab$V1 <- sub("^", "0", metab$public_client_id)
#filter samples based on those that have microbiome data
ids<-c(sample_data(df)$public_client_id)
metab<-metab %>% filter(V1 %in% ids)
metab<-metab[order(metab$public_client_id),]
dim(metab)

In [ ]:
#save metadata to csv
write.csv(metab,'entero_df.csv')

In [ ]:
#generate a genus level table with genus as names
OTU_df = as(otu_table(df), "matrix")
# transpose if necessary
if(taxa_are_rows(df)){OTU_df <- t(OTU_df)}
# Coerce to data.frame
OTU_df = as.data.frame(OTU_df)
colnames(OTU_df)<-c(tax_table(df)[,6])
#add enterotype and index columns
OTU_df$entero<-sample_data(df)$entero
OTU_df$public_client_id<-sample_data(df)$public_client_id
write.csv(OTU_df,"genus_table.csv")

In [ ]:
#create relative abundance genus tables ploting major taxa across enterotypes
OTU_df = as(otu_table(df), "matrix")/sample_sums(df)
# transpose if necessary
if(taxa_are_rows(df)){OTU_df <- t(OTU_df)}
# Coerce to data.frame
OTU_df = as.data.frame(OTU_df)
colnames(OTU_df)<-c(tax_table(df)[,6])
OTU_df$entero<-sample_data(df)$entero
OTU_df$public_client_id<-sample_data(df)$public_client_id
#plot major taxa shown in figure 3
color_scheme=c("orange2", "slategray1", "steelblue4","yellowgreen")
p = ggplot(OTU_df, aes(entero,Faecalibacterium,fill=factor(entero))) + theme_bw()+ scale_fill_manual(values = color_scheme)+stat_boxplot(geom = "errorbar", width = .5)+geom_boxplot(outlier.size = .4)+theme(text=element_text(size=14))
p+theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))
ggsave("faecalibacterium.pdf", width = 4, height = 4)
p = ggplot(OTU_df, aes(entero,Bacteroides,fill=factor(entero)))
p+theme_bw()+scale_fill_manual(values = color_scheme)+stat_boxplot(geom = "errorbar", width = .5)+geom_boxplot(outlier.size = .4)+theme(text=element_text(size=14))+theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))
ggsave("Bacteroides.pdf", width = 4, height = 4)
p = ggplot(OTU_df, aes(entero,Prevotella_9,fill=factor(entero)))
p+theme_bw()+scale_fill_manual(values = color_scheme)+stat_boxplot(geom = "errorbar", width = .5)+geom_boxplot(outlier.size = .4)+theme(text=element_text(size=14))+theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))
ggsave("prevotella.pdf", width = 4, height = 4)

In [ ]:
#generate a Bray-Curtis PCoA plot color-coded by enterotype (Fig.3A)
GP.ord <- ordinate(df, "PCoA", "bray")
p1 = plot_ordination(df, GP.ord, type="samples", color="entero", title="enterotypes")
color_scheme=c("orange2", "slategray1", "steelblue4","yellowgreen")
#p1+theme_bw()+scale_fill_manual(values=color_scheme)+theme(text=element_text(size=14))+geom_point(size=3)+geom_point(shape = 1,size = 3.0,colour = "black",stroke=0.25)
p1+theme_bw()+scale_color_manual(values = color_scheme)+theme(text=element_text(size=18))+geom_point(size=3)+geom_point(shape = 1,size = 3.0,colour = "black",stroke=0.1)
ggsave("PCoA_entero.pdf", width = 8, height = 6)